# Exploratory Data Analysis - EDA

In [ ]:
import os 
import time
import torch
import math
import numpy as np
import pandas as pd
import torch.nn as nn
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from torch.utils.data import Dataset, DataLoader

hn_data = pd.read_csv(r"//media/khanhxoe/New Volume/Studying/Project_HUST/Finished Project/Impute_misvalues_hanoi.csv", usecols=[0,1,2])
hn_data['Date'] = hn_data['Date'].ffill()
hn_data['Date'] = pd.to_datetime(hn_data['Date'])
hn_data = hn_data.set_index('Date', drop= True)
hn_data['Day'] = hn_data.index.day
hn_data['Month'] = hn_data.index.month

In [ ]:
def check_dataframe_health(df):
    print(" Checking for NaNs...")
    print(df.isna().sum(), "\n")

    print(" Checking for non-numeric values...")
    for col in df.columns:
        try:
            pd.to_numeric(df[col])
        except Exception as e:
            print(f"⚠️ Column '{col}' has non-numeric values: {e}")

    print("\n Checking column data types:")
    print(df.dtypes)


    print("\n Checking value ranges (summary stats):")
    print(df.describe(include='all'))

    print("\n Done checking.")

# Gọi hàm với DataFrame của ông
check_dataframe_health(hn_data)

def find_weird_values(df):
    for col in df.columns:
        weird_vals = df[col][~df[col].apply(lambda x: str(x).replace('.', '', 1).isdigit())]
        if not weird_vals.empty:
            print(f"Weird values in '{col}':")
            print(weird_vals.unique())  # in thử vài cái thôi cho đỡ rối
            print()

find_weird_values(hn_data)

In [ ]:
hn_data['Hour'] = hn_data['Hour'].replace('#NUM!', 0)
hn_data['Hour'] = hn_data['Hour'].astype(int)

In [ ]:
sample = hn_data[(hn_data.index.year == 2017) & ((hn_data.index.month == 7)) & ((hn_data.index.day == 29))]
plt.figure(figsize= (25, 5))
plt.plot(sample['Waterlevel'].values)

# Build DataLoader

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset

class Series_Dataset(Dataset):
    def __init__(self, data, past_len, pred_len, stride=1):
        super(Series_Dataset, self).__init__()
        self.past_len = past_len
        self.pred_len = pred_len
        self.stride = stride
        self.df = data

        self.x_values, self.y_shifted, self.y_label, self.x_times, self.y_shifted_times, self.y_label_times = \
            self.create_sequences_with_cores_time(self.df, self.past_len, self.pred_len)

    def create_sequences_with_cores_time(
            self, data: pd.DataFrame, 
            past_len: int, pred_len: int
        ):
        
        x_values, y_shifted, y_label = [], [], []
        x_hours, y_shifted_hours, y_label_hours = [], [], []
        x_days, y_shifted_days, y_label_days = [], [], []
        x_months, y_shifted_months, y_label_months = [], [], []

        for i in range(0, len(data) - past_len - pred_len + 1):
            x_seq = data.iloc[i:i + past_len]
            y_shifted_seq = data.iloc[i+past_len-1 : i+past_len+pred_len-1]
            y_label_seq = data.iloc[i + past_len:i + past_len + pred_len]

            # Values
            x_values.append(x_seq['Waterlevel'].values)
            y_shifted.append(y_shifted_seq['Waterlevel'].values)
            y_label.append(y_label_seq['Waterlevel'].values)

            # Time features
            x_hours.append(x_seq['Hour'].values)
            y_shifted_hours.append(y_shifted_seq['Hour'].values)
            y_label_hours.append(y_label_seq['Hour'].values)

            x_days.append(x_seq['Day'].values)
            y_shifted_days.append(y_shifted_seq['Day'].values)
            y_label_days.append(y_label_seq['Day'].values)

            x_months.append(x_seq['Month'].values)
            y_shifted_months.append(y_shifted_seq['Month'].values)
            y_label_months.append(y_label_seq['Month'].values)

        x_values = np.array(x_values)
        y_shifted = np.array(y_shifted)
        y_label = np.array(y_label)

        x_times = np.stack([
            np.array(x_hours, dtype=np.int32),
            np.array(x_days, dtype=np.int32),
            np.array(x_months, dtype=np.int32)
        ], axis=-1)

        y_shifted_times = np.stack([
            np.array(y_shifted_hours, dtype=np.int32),
            np.array(y_shifted_days, dtype=np.int32),
            np.array(y_shifted_months, dtype=np.int32)
        ], axis=-1)

        y_label_times = np.stack([
            np.array(y_label_hours, dtype=np.int32),
            np.array(y_label_days, dtype=np.int32),
            np.array(y_label_months, dtype=np.int32)
        ], axis=-1)
        
        return (
            torch.tensor(x_values, dtype=torch.float32),
            torch.tensor(y_shifted, dtype=torch.float32),
            torch.tensor(y_label, dtype=torch.float32),
            torch.tensor(x_times, dtype=torch.int),
            torch.tensor(y_shifted_times, dtype=torch.int),
            torch.tensor(y_label_times, dtype=torch.int)
        )

    def __len__(self):
        return len(self.x_values)

    def __getitem__(self, idx):
        x_values = self.x_values[idx].unsqueeze(-1)
        y_shifted = self.y_shifted[idx].unsqueeze(-1)
        y_label = self.y_label[idx].unsqueeze(-1)
        
        x_times = self.x_times[idx]
        y_shifted_times = self.y_shifted_times[idx]
        y_label_times = self.y_label_times[idx]

        return x_values, y_shifted, y_label, x_times, y_shifted_times, y_label_times

**Data Ha Noi & Vu Quang**

In [ ]:
train_data = hn_data.loc[(hn_data.index.year >= 2008) & (hn_data.index.year <= 2016)]
test_data = hn_data.loc[(hn_data.index.year >= 2017) & (hn_data.index.year <= 2017)]

**Data Hung Yen**

In [ ]:
train_data = hn_data.loc[(hn_data.index.year >= 2008) & (hn_data.index.year <= 2013)]
test_data = hn_data.loc[(hn_data.index.year >= 2014) & (hn_data.index.year <= 2015)]

## Split Data

*Splitting data into 2 dataset: drought dataset and flood dataset*

In [ ]:
drought_test_data = hn_data.loc[
    (hn_data.index.year == 2017) &
    (
        (hn_data.index.month >= 1) &
        (hn_data.index.month <= 3)
    ) 
]

flood_test_data = hn_data.loc[
    (hn_data.index.year == 2017) &
    ((hn_data.index.month >=6) & (hn_data.index.month <=8))
]

*Data values need to be scaled due to the sensitivity of Transformer*

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_data.loc[:, 'Waterlevel'] = scaler.fit_transform(train_data[['Waterlevel']])
test_data.loc[:, 'Waterlevel'] = scaler.transform(test_data[['Waterlevel']])
drought_test_data.loc[:, 'Waterlevel'] = scaler.transform(drought_test_data[['Waterlevel']])
flood_test_data.loc[:, 'Waterlevel'] = scaler.transform(flood_test_data[['Waterlevel']])

# Model Architecture

## Xây dựng các mốc thời gian, nhúng cái mốc dữ liệu 

**Tạo dấu mốc thời gian cho các vị trí nằm trong chuỗi dữ liệu**

In [ ]:
class PositionalInputEmbedding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalInputEmbedding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # [1, max_len, d_model]
        self.register_buffer('pe', pe)

    def forward(self, x):  # x: [B, L, D] or [B, L, input_dim]
        return self.pe[:, :x.size(1), :].expand(x.size(0), -1, -1)  # [B, L, D]

In [ ]:
class ValueEmbedding(nn.Module):
    def __init__(self, input_dim, d_model, device):
        super(ValueEmbedding, self).__init__()
        
        self.fc = nn.Linear(input_dim, d_model, device=device)

    def forward(self, x):
        x = self.fc(x)
        return x

**Nhúng các móc thời gian vào chiều không gian lớn hơn**

In [ ]:
import torch.nn.init as init

class TemporalEmbedding(nn.Module):
    def __init__(self, d_model):
        super(TemporalEmbedding, self).__init__()
        
        self.device = "cuda"
        
        self.hour_list = torch.tensor([1,4,7,10,13,16,19,22]).to(self.device)
        self.day_list = torch.tensor(np.arange(1, 32)).to(self.device)
        self.month_list = torch.tensor(np.arange(1, 13)).to(self.device)
        
        self.hour_embed = nn.Embedding(len(self.hour_list), d_model)
        self.day_embed = nn.Embedding(len(self.day_list), d_model)
        self.month_embed = nn.Embedding(len(self.month_list), d_model)

        #init.xavier_uniform_(self.hour_embed.weight)
        #init.xavier_uniform_(self.day_embed.weight)
        #init.xavier_uniform_(self.month_embed.weight)
        
        self.hour_norm = nn.LayerNorm(d_model)
        self.day_norm = nn.LayerNorm(d_model)
        self.month_norm = nn.LayerNorm(d_model)

        self.temporal_proj = nn.Linear(d_model * 2, d_model)

    def forward(self, x_mark):
        
        x_hours = x_mark[:, :, 0].reshape(x_mark.shape[0], -1).to(self.device)
        x_days = x_mark[:, :, 1].reshape(x_mark.shape[0], -1).to(self.device)
        x_months = x_mark[:, :, 2].reshape(x_mark.shape[0], -1).to(self.device)

        hour_indices = (x_hours.unsqueeze(-1) == self.hour_list ).float().argmax(dim=-1).to(self.device)
        day_indices = (x_days.unsqueeze(-1) == self.day_list).float().argmax(dim=-1).to(self.device)
        month_indices = (x_months.unsqueeze(-1) == self.month_list).float().argmax(dim=-1).to(self.device)

        hour_x = self.hour_norm(self.hour_embed(hour_indices))
        day_x = self.day_norm(self.day_embed(day_indices))
        month_x = self.month_norm(self.month_embed(month_indices))
        
        out = torch.cat([day_x, month_x], dim=-1)  # shape: [B, T, 3*d_model]
        out = self.temporal_proj(out)     

        return out.to(self.device)

**Xử lý nhúng dữ liệu**

In [ ]:
class DataEmbedding(nn.Module):
    def __init__(self, input_dim, d_model, device, dropout=0.1):
        super(DataEmbedding, self).__init__()

        self.device = device
        
        self.value_embedding = ValueEmbedding(input_dim, d_model, device= device)
        self.position_embedding = PositionalInputEmbedding(d_model)
        self.temporal_embedding = TemporalEmbedding(d_model)
        self.dropout = nn.Dropout(dropout)

        self.prj = nn.Linear(
            in_features = d_model*3,
            out_features = d_model
        )
        self.prj2 = nn.Linear(
            in_features= d_model,
            out_features= d_model
        )
    
    def forward(self, x, x_times):
        value_emb = self.value_embedding(x)              
        pos_emb = self.position_embedding(x)             
        temp_emb = self.temporal_embedding(x_times)         
        x = torch.cat([value_emb, pos_emb, temp_emb], dim=-1)
        #x = value_emb + pos_emb + temp_emb
        x = self.prj(x)                                        
    
        return self.dropout(x).to(self.device)

## Kiến trúc mô hình Transformer

**Convolutional Attention**

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model phải chia hết cho num_heads"
        self.kernel_size = 7

        self.conv_q = nn.Conv1d(d_model, d_model, kernel_size= self.kernel_size, padding=(self.kernel_size-1)//2, bias=False)
        self.conv_k = nn.Conv1d(d_model, d_model, kernel_size= self.kernel_size, padding=(self.kernel_size-1)//2, bias=False)

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        
        self.out_linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value, mask=None):
        batch_size = query.shape[0]
        q = self.conv_q(query.permute(0, 2, 1)).permute(0, 2, 1)
        k = self.conv_k(key.permute(0, 2, 1)).permute(0, 2, 1)

        q = self.q_linear(query)
        k = self.k_linear(key)
        v = self.v_linear(value)
        
        def reshape(x):
            return x.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)

        q, k, v = map(reshape, (q, k, v))
        self.scale = torch.tensor(1.0 / math.sqrt(self.d_k))
        attn_scores = torch.matmul(q, k.transpose(-2, -1)) * self.scale

        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask, float('-inf'))

        attn_probs = F.softmax(attn_scores, dim=-1) 
        attn_probs = self.dropout(attn_probs)
        output = torch.matmul(attn_probs, v)
        output = output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)

        return self.out_linear(output)

**FFN**

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(FeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.dropout(self.relu(self.fc1(x))))

**EncoderLayers**

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.kernel_size = 3

        self.attn = MultiHeadAttention(
            d_model=d_model, 
            num_heads=num_heads,
            dropout=dropout
        )
        
        self.ffn = FeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        self.omega = nn.Parameter(torch.ones(1, 1, d_model))
        nn.init.xavier_normal_(self.omega, gain=1.0)
        self.omega2 = nn.Parameter(torch.ones(1, 1, d_model))
        nn.init.xavier_normal_(self.omega2, gain=1.0)

    def forward(self, x, mask=None):
        shortcut = x * self.omega
        x = self.norm1(x)
        attn_output = self.attn(x, x, x, mask) 
        attn_output = self.dropout(attn_output) 
        attn_output = attn_output + shortcut

        shortcut2 = attn_output * self.omega2
        attn_output = self.norm1(attn_output)
        ffn_output = self.dropout(self.ffn(attn_output))
        ffn_output = ffn_output + shortcut2
        return self.norm2(ffn_output)

**Encoder**

In [ ]:
class Encoder(nn.Module):
    def __init__(
        self, pred_dim, 
        d_model, num_layers,
        num_heads, d_ff, 
        device, dropout=0.1
        ):
        super(Encoder, self).__init__()
        
        self.embedding = DataEmbedding(pred_dim, d_model, device, dropout)
        self.layers = nn.ModuleList([
            EncoderLayer(d_model, num_heads, d_ff, dropout) 
            for _ in range(num_layers)
        ])
        
        self.norm = nn.LayerNorm(d_model, device= device)

    def forward(self, x, x_times, mask=None):
        x = self.embedding(x, x_times)
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

**DecoderLayers**

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(DecoderLayer, self).__init__()
        
        self.self_attn = MultiHeadAttention(
            d_model, num_heads, dropout
        )
        
        self.cross_attn = MultiHeadAttention(
            d_model, num_heads, dropout
        )
        
        self.ffn = FeedForward(d_model, d_ff, dropout)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, tgt_mask=None, src_mask=None):
        x_norm = self.norm1(x)
        self_attn_output = self.self_attn(x_norm, x_norm, x_norm, tgt_mask)
        x = x + self.dropout(self_attn_output)

        x_norm = self.norm2(x)
        cross_attn_output = self.cross_attn(x_norm, enc_output, enc_output, src_mask)
        x = x + self.dropout(cross_attn_output)
        
        x_norm = self.norm3(x)
        ffn_output = self.ffn(x_norm)
        x = x + self.dropout(ffn_output)

        return x

**Decoder**

In [ ]:
class Decoder(nn.Module):
    def __init__(self, tar_dim, d_model, num_layers, num_heads, d_ff, device, dropout=0.1):
        super(Decoder, self).__init__()
        self.embedding = DataEmbedding(tar_dim, d_model, device, dropout)
        self.layers = nn.ModuleList([
            DecoderLayer(d_model, num_heads, d_ff, dropout) 
            for _ in range(num_layers)
        ])
        
        self.norm = nn.LayerNorm(d_model, device= device)

    def forward(self, x, x_times, enc_output, tgt_mask= None, src_mask= None):
        x = self.embedding(x, x_times)
        for layer in self.layers:
            x = layer(x, enc_output, tgt_mask, src_mask)
        return self.norm(x)

**Transformer**

In [ ]:
class Transformer(nn.Module):
    def __init__(self, pred_dim, tar_dim, device, d_model=512, num_layers=6, num_heads=8, d_ff=2048, dropout=0.1):
        super(Transformer, self).__init__()
        
        self.device = device
        self.encoder = Encoder(pred_dim, d_model, num_layers, num_heads, d_ff, device, dropout)
        self.decoder = Decoder(tar_dim, d_model, num_layers, num_heads, d_ff, device, dropout)
        self.out = nn.Linear(d_model, tar_dim)

    def generate_mask(self, x_values, y_values):
        batch_size = x_values.size(0)
        pred_len = y_values.size(1)

        past_mask = None
        pred_mask = torch.triu(
            torch.ones((pred_len, pred_len), device=self.device), diagonal=1
        ).bool()  

        pred_mask = pred_mask.unsqueeze(0).unsqueeze(1)  # [1, 1, tgt_seq_len, tgt_seq_len]
        pred_mask = pred_mask.expand(batch_size, 1, -1, -1)  # [batch_size, 1, tgt_seq_len, tgt_seq_len]

        return past_mask, pred_mask
    
    def forward(self, x_values, x_times, y_shifted, y_shifted_times, past_mask=None, pred_mask=None): 
        past_mask, pred_mask = self.generate_mask(x_values, y_shifted)
        enc_output = self.encoder(x_values, x_times, past_mask)
        dec_output = self.decoder(y_shifted, y_shifted_times, enc_output, pred_mask, past_mask) 
        return self.out(dec_output)

    def inference_reusing(self, x_values, x_times, y_label_times):

        self.eval()
        batch_size = x_values.size(0)
        pred_len = y_label_times.size(1)
        past_mask = None
        y_pred = []

        enc_output = self.encoder(x_values, x_times, past_mask)
        dec_input = x_values[:, -1:, :]
        dec_time_input = x_times[:, -1:, :]

        for i in range(pred_len):
            _, pred_mask = self.generate_mask(x_values, dec_input)
            dec_output = self.decoder(dec_input, dec_time_input, enc_output, pred_mask, past_mask)
            pred = self.out(dec_output)

            dec_input = torch.concat([dec_input, pred[:, -1:, :]], dim=1)
            dec_time_input = torch.concat([dec_time_input, y_label_times[:, i:i+1, :]], dim=1)

        return dec_input[:, 1:, :]

# Training loop

**Building the dataloader**

In [ ]:
batch_size = 32
seq_len = 40
pred_len = 24
num_workers = 6
pin_mem = True

train_dataset = Series_Dataset(train_data, past_len=seq_len, pred_len=pred_len, stride=1)
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=pin_mem,
    drop_last=True
)
test_dataset = Series_Dataset(test_data, past_len=seq_len, pred_len=pred_len, stride=1)
test_loader = DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=pin_mem,
    drop_last=True
)
drought_dataset = Series_Dataset(drought_test_data, past_len=seq_len, pred_len=pred_len, stride=1)
drought_loader = DataLoader(
    drought_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=pin_mem
)
flood_test_dataset = Series_Dataset(flood_test_data, past_len=seq_len, pred_len=pred_len, stride=1)
flood_loader = DataLoader(
    flood_test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=pin_mem
)

**Building the training and evaluating function**

In [ ]:
def evaluate(model, data_loader, criterion, device):
    model.eval()
    losses = []
    with torch.no_grad():
        for batch in data_loader:        
            x_values = batch[0].to(device)
            y_shifted = batch[1].to(device)
            y_label = batch[2].to(device)
    
            x_times = batch[3].to(device)
            y_shifted_times = batch[4].to(device)
            y_label_times = batch[5].to(device)

            output = model(x_values, x_times, y_shifted, y_shifted_times) 
            loss = criterion(output.squeeze(-1), y_label.squeeze(-1))
            
            losses.append(loss.item())
    
    return sum(losses) / len(losses)

def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    epoch_losses = []
    
    for batch in train_loader:
        optimizer.zero_grad()
        
        x_values = batch[0].to(device)
        y_shifted = batch[1].to(device)
        y_label = batch[2].to(device)

        x_times = batch[3].to(device)
        y_shifted_times = batch[4].to(device)
        y_label_times = batch[5].to(device)

        output = model(x_values, x_times, y_shifted, y_shifted_times) 
        loss = criterion(output.squeeze(-1), y_label.squeeze(-1))
        
        loss.backward()
        optimizer.step()
        
        epoch_losses.append(loss.item())
    
    return sum(epoch_losses) / len(epoch_losses)
def fit(model, train_loader, test_loader, criterion, optimizer, num_epochs, device, scheduler=None):
    train_losses = []
    val_losses = []
    for epoch in range(num_epochs):
        start_time = time.time()
        
        train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
        train_losses.append(train_loss)
        
        val_loss = evaluate(model, test_loader, criterion, device)
        val_losses.append(val_loss)
        
        if scheduler is not None:
            scheduler.step(val_loss)
        
        epoch_time = time.time() - start_time
        
        current_lr = optimizer.param_groups[0]['lr']
        if current_lr <= 5e-7:
            return train_losses, val_losses
        
        print(f'Epoch [{epoch+1}/{num_epochs}] | '
              f'Train Loss: {train_loss:.6f} | '
              f'Time: {epoch_time:.2f}s | '
              f'Val Loss: {val_loss:.6f}')
    
    return train_losses, val_losses

**Initiate Transformer model**

In [ ]:
seq_len = 40
pred_len = 24
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Transformer(
    pred_dim= 1, tar_dim= 1, 
    d_model= 256, num_layers= 6, 
    num_heads= 8, d_ff= 1024, 
    device= device, dropout= 0.1
).to(device)

**Training loop**

In [ ]:
criterion = nn.HuberLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-2, weight_decay=1e-9)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=4)
train_losses, val_losses = fit(model, train_loader, test_loader, criterion, optimizer, num_epochs=70, device=device, scheduler=scheduler)

# Evaluation

## Standard random evaluation for recursive multi-step forecasting

In [ ]:
import numpy as np
import torch
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import math

def sim_score(x, y):
    y = np.array(y).flatten()
    x = np.array(x).flatten()
    
    T = len(y)
    diff = np.abs(y - x)
    denom = np.max(x) - np.min(x)
    
    if denom == 0:
        return 0.0

    sim = np.sum(1 / (1 + (diff / denom))) / T
    return sim

def compute_metrics(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()

    mae = mean_absolute_error(y_true, y_pred)
    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    sim = sim_score(y_true, y_pred)

    std_y = np.std(y_true)
    std_x = np.std(y_pred)
    fsd = 0.0 if std_y + std_x == 0 else 2 * abs(std_y - std_x) / (std_y + std_x)

    return {
        "Sim": sim,
        "MAE": mae,
        "RMSE": rmse,
        "R2": r2,
        "FSD": fsd
    }

def eval_random(model, test_df, num_samples, seq_len, pred_len, device, target_scaler=None, verbose=False, top_k=None):
    model.eval()
    max_index = len(test_df) - seq_len - pred_len + 1 
    
    scores = []
    rand_idx = np.random.randint(0, max_index, size=num_samples)    

    x_values, x_times, y_label_times, y_labels = [], [], [], []

    for i in rand_idx:
        x_value = test_df.iloc[i:i+seq_len]['Waterlevel'].values
        x_time = test_df.iloc[i:i+seq_len][['Hour', 'Day', 'Month']].values
        y_label = test_df.iloc[i+seq_len:i+seq_len+pred_len]['Waterlevel'].values
        y_label_time = test_df.iloc[i+seq_len:i+seq_len+pred_len][['Hour', 'Day', 'Month']].values

        x_values.append(torch.tensor(x_value, dtype=torch.float32).unsqueeze(-1))
        x_times.append(torch.tensor(x_time, dtype=torch.int))
        y_labels.append(torch.tensor(y_label, dtype=torch.float32).unsqueeze(-1))
        y_label_times.append(torch.tensor(y_label_time, dtype=torch.int))

    x_values = torch.stack(x_values).to(device)       # [B, seq_len, 1]
    x_times = torch.stack(x_times).to(device)         # [B, seq_len, 3]
    y_label_times = torch.stack(y_label_times).to(device)  # [B, pred_len, 3]
    y_labels = torch.stack(y_labels).to(device)       # [B, pred_len, 1]

    with torch.no_grad():
        y_preds = model.inference_reusing(x_values, x_times, y_label_times)  # [B, pred_len, 1]

        y_preds_np = y_preds.cpu().numpy().reshape(num_samples * pred_len, 1)
        y_labels_np = y_labels.cpu().numpy().reshape(num_samples * pred_len, 1)

        if target_scaler:
            y_preds_np = target_scaler.inverse_transform(y_preds_np)
            y_labels_np = target_scaler.inverse_transform(y_labels_np)

        y_preds_np = y_preds_np.reshape(num_samples, pred_len, 1)
        y_labels_np = y_labels_np.reshape(num_samples, pred_len, 1)

        for i in range(num_samples):
            score = compute_metrics(y_labels_np[i], y_preds_np[i])
            score["index"] = rand_idx[i]
            scores.append(score)

        scores_sorted = sorted(scores, key=lambda x: x["RMSE"])
        k = top_k if top_k else num_samples
        top_scores = scores_sorted[:k]

        return top_scores

def average_scores(score_list1, score_list2):
    assert len(score_list1) == len(score_list2), "Hai danh sách phải có cùng số lượng mẫu"

    metrics = ["Sim", "MAE", "RMSE", "R2", "FSD"]
    avg_results = {}

    for metric in metrics:
        values1 = [s[metric] for s in score_list1]
        values2 = [s[metric] for s in score_list2]

        avg_val = (np.mean(values1) + np.mean(values2)) / 2
        avg_results[metric] = avg_val

    return avg_results

**Eval samples**

In [ ]:

top_results1 = eval_random(
    model, drought_test_data,
    num_samples=20,
    seq_len=seq_len,
    pred_len=4,
    device=device,
    target_scaler=scaler,
    verbose=True,
    top_k=10
)

top_results2 = eval_random(
    model, flood_test_data,
    num_samples=20,
    seq_len=seq_len,
    pred_len=4,
    device=device,
    target_scaler=scaler,
    verbose=True,
    top_k=10
)
avg_metrics = average_scores(top_results1, top_results2)
print("=== Trung bình giữa 2 tập test ===")
print(" ".join([f"{value:.4f} " for metric, value in avg_metrics.items()]))

# Draw line

In [ ]:
def plot_multi_models_forecast(
    models, model_names, model_types,
    test_df, index, seq_len,
    pred_len, device, target_scaler=None,
    time_cols=['Hour', 'Day', 'Month'],
    color_list=None,
    marker_list=None
):
    if color_list is None:
        color_list = ['red', 'orange', 'purple', 'brown', 'cyan', 'magenta']
    if marker_list is None:
        marker_list = ['o', 's', 'D', '^', 'v', '*']

    x_value_raw = test_df.iloc[index:index+seq_len]['Waterlevel'].values
    x_time = test_df.iloc[index:index+seq_len][time_cols].values
    y_label_raw = test_df.iloc[index+seq_len:index+seq_len+pred_len]['Waterlevel'].values
    y_label_time = test_df.iloc[index+seq_len:index+seq_len+pred_len][time_cols].values

    if target_scaler is not None:
        y_label_raw = target_scaler.inverse_transform(y_label_raw.reshape(-1, 1)).flatten()

    x_times = torch.tensor(x_time, dtype=torch.int64).unsqueeze(0).to(device)
    y_label_times = torch.tensor(y_label_time, dtype=torch.int64).unsqueeze(0).to(device)

    plt.figure(figsize=(12, 12))
    t_input = np.arange(seq_len)
    t_pred = np.arange(0, pred_len)

    if target_scaler is not None:
        x_value_raw_plot = target_scaler.inverse_transform(x_value_raw.reshape(-1, 1)).flatten()
    else:
        x_value_raw_plot = x_value_raw

    #plt.plot(t_input, x_value_raw_plot, label="Input (Past)", color='black')
    plt.plot(t_pred, y_label_raw, label="Ground Truth", color='green')

    for i, model in enumerate(models):
        model_type = model_types[i]
        model_name = model_names[i]
        color = color_list[i % len(color_list)]
        marker = marker_list[i % len(marker_list)]

        if model_type == "transformer":
            x_tensor = torch.tensor(x_value_raw, dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(device)
        elif model_type == "lstm" or model_type == "dlstm":
            if target_scaler is not None:
                x_value = target_scaler.inverse_transform(x_value_raw.reshape(-1, 1)).flatten()
            else:
                x_value = x_value_raw
            x_tensor = torch.tensor(x_value, dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(device)
        else:
            raise ValueError(f"Model type không hợp lệ: {model_type}")

        model.eval()
        with torch.no_grad():
            if model_type == "transformer":
                y_pred = model.inference_reusing(x_tensor, x_times, y_label_times)
                y_pred_np = y_pred.squeeze().cpu().numpy()
                if target_scaler:
                    y_pred_np = target_scaler.inverse_transform(y_pred_np.reshape(-1, 1)).flatten()
                #plt.plot(t_pred, y_pred_np, label=model_name, color=color, marker=marker, linestyle='-', markersize=4)                    
            
            elif model_type == "lstm":
                y_pred = model.predict_AR_multi_step_ahead(x_tensor, pred_len)
                y_pred_np = y_pred.squeeze().cpu().numpy()
            elif model_type == "dlstm":
                y_pred = model(x_tensor)
                y_pred_np = y_pred.squeeze().cpu().numpy()

        plt.plot(t_pred, y_pred_np, label=model_name, color=color, marker=marker, linestyle='-', markersize=4)                

    plt.axvline(x=seq_len-1, color='gray', linestyle=':', label='Prediction Start')
    plt.xlabel("Timestep")
    plt.ylabel("Waterlevel")
    plt.ylim(0, 400)
    plt.title("Multi-step Forecasting - All Models")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

plot_multi_models_forecast(
    models=[model1, model2, model3, model1_1, model2_1, model3_1, model4, model5, model6],
    model_names=["lstm1", "lstm2", "lstm3", "lstm", "lstm","lstm","Trans1", "Trans2", "Trans3"],
    model_types=["lstm", "lstm", "lstm", "dlstm", "dlstm","dlstm", "transformer", "transformer", "transformer"],
    test_df=drought_test_data,
    index=100,
    seq_len=16,
    pred_len= 40,
    device=device,
    target_scaler=scaler
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from itertools import cycle

def plot_multi_models_forecast(
    models, model_names, model_types,
    test_df, index, seq_len,
    pred_len, plot_len, device, target_scaler=None,
    time_cols=['Hour', 'Day', 'Month'],
    color_list=None, marker_list=None,
    random_colors=False,
    save_path=None
):
    if random_colors:
        color_list = [plt.cm.tab20(i) for i in range(len(models))]
    elif color_list is None or len(color_list) < len(models):
        default_colors = [
            '#e6194b',  # Red
            '#3cb44b',  # Green
            "#99b400",  # Blue
            "#a85417",  # Orange (darker)
            "#5e1974",  # Purple
            "#1b2fa5",  # Cyan (neon-like)
            '#f032e6',  # Magenta
            "#057025",  # Lime green
            '#fabebe',  # Light pink
            '#008080'   # Teal
            ]

        color_list = (default_colors * ((len(models) // len(default_colors)) + 1))[:len(models)]

    if marker_list is None or len(marker_list) < len(models):
        default_markers = ['o', 's', 'D', '^', 'v', '*', 'x', 'P', '<', '>']
        marker_list = (default_markers * ((len(models) // len(default_markers)) + 1))[:len(models)]

    # Lấy dữ liệu input và ground truth
    x_value_raw = test_df.iloc[index:index+seq_len]['Waterlevel'].values
    x_time = test_df.iloc[index:index+seq_len][time_cols].values
    y_label_raw = test_df.iloc[index+seq_len:index+seq_len+pred_len]['Waterlevel'].values
    y_label_time = test_df.iloc[index+seq_len:index+seq_len+pred_len][time_cols].values

    if target_scaler is not None:
        y_label_raw = target_scaler.inverse_transform(y_label_raw.reshape(-1, 1)).flatten()

    x_times = torch.tensor(x_time, dtype=torch.int64).unsqueeze(0).to(device)
    y_label_times = torch.tensor(y_label_time, dtype=torch.int64).unsqueeze(0).to(device)

    plt.figure(figsize=(8, 10))
    t_pred = np.arange(pred_len)

    plt.plot(t_pred[:plot_len], y_label_raw[:plot_len], label="Ground Truth", color='green', linewidth=2.5)

    for i, model in enumerate(models):
        model_type = model_types[i]
        model_name = model_names[i]
        color = color_list[i]
        marker = marker_list[i]

        if model_type == "transformer":
            x_tensor = torch.tensor(x_value_raw, dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(device)
        elif model_type == "lstm" or model_type == "dlstm":
            if target_scaler is not None:
                x_value = target_scaler.inverse_transform(x_value_raw.reshape(-1, 1)).flatten()
            else:
                x_value = x_value_raw
            x_tensor = torch.tensor(x_value, dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(device)
        else:
            raise ValueError(f"Model type không hợp lệ: {model_type}")

        model.eval()
        with torch.no_grad():
            if model_type == "transformer":
                y_pred = model.inference_reusing(x_tensor, x_times, y_label_times)
                y_pred_np = y_pred.squeeze().cpu().numpy()
                if target_scaler:
                    y_pred_np = target_scaler.inverse_transform(y_pred_np.reshape(-1, 1)).flatten()        
            
            elif model_type == "lstm":
                y_pred = model.predict_AR_multi_step_ahead(x_tensor, pred_len)
                y_pred_np = y_pred.squeeze().cpu().numpy()
            elif model_type == "dlstm":
                y_pred = model(x_tensor)
                y_pred_np = y_pred.squeeze().cpu().numpy()

        plt.plot(
            t_pred[:plot_len],
            y_pred_np[:plot_len],
            label=model_name,
            color=color,
            marker=marker,
            linestyle='-',
            linewidth=1,
            markersize=5
        )

    plt.axvline(x=0, color='gray', linestyle=':', label='Prediction Start')
    plt.xlabel("Prediction Step", fontsize=12)
    plt.ylabel("Water Level", fontsize=12)
    plt.title("Multi-step Forecasting - All Models", fontsize=16)
    plt.legend(fontsize=10, loc='upper right')
    plt.ylim(50, 300)
    plt.grid(True)
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=300)
        print(f"✅ Saved figure to {save_path}")

    plt.show()

plot_multi_models_forecast(
    models=[model1, model2, model3, model1_1, model2_1, model3_1, model4, model5, model6],
    model_names=["direct_lstm1", "direct_lstm2", "direct_lstm3", "auto_lstm1", "auto_lstm2","auto_lstm3", "Trans1", "", "Trans3"],
    model_types=["lstm", "lstm", "lstm", "dlstm", "dlstm","dlstm", "transformer", "transformer", "transformer"],
    test_df=drought_test_data,
    index=100,
    seq_len=16,
    pred_len=40,
    plot_len= 20,
    device=device,
    target_scaler=scaler,
    random_colors=False,
    save_path=None
)
